<a href="https://colab.research.google.com/github/nineto1/final_project/blob/main/train_StyleGAN2_ADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Clone 'StyleGAN2-ADA' Repository and go inside the directory

- Colab 에서 GPU를 할당받아 사용하고 있는지 확인

In [ ]:
!nvidia-smi

Sun Jan 16 05:49:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

- StyleGAN2-ADA는 Tensorflow 2.x 버전을 지원하지 않습니다. 따라서 Tensorflow 1.X 버전을 사용하도록 설정해주어야 합니다.

In [ ]:
# TensorFlow 2.x is not supported in 'StyleGAN2-ADA'
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# clone 'StyleGAN2-ADA'
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd /content/stylegan2-ada

Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 74 (delta 0), reused 1 (delta 0), pack-reused 71
Unpacking objects: 100% (74/74), done.
/content/stylegan2-ada


## 2. Preparing Dataset


- Mounted Google drive

In [ ]:
# Mounted Google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


- Convert our dataset(IMAGE) to '.tfrecords' format
  - 원본 이미지 파일이 '.jpg' 형태로 저장되어 있기 때문에, Tensorflow에서 활용할 수 있도록 '.tfrecords' 형태로 변환하여 줍니다.

In [ ]:
# Load own dataset from google drive
# path = {구글 드라이브 내 데이터셋 경로}
path = "/content/drive/MyDrive/styleGAN/celeb_face"

path_dataset = "./dataset/Img" # .tfrecords 파일이 저장될 경로

In [ ]:
!python dataset_tool.py create_from_images {path_dataset} {path}

Loading images from "/content/drive/MyDrive/styleGAN/celeb_face"
Creating dataset "./dataset/Img"
dataset_tool.py:96: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 9773 images.


In [ ]:
!ls {path_dataset}

Img-r02.tfrecords  Img-r04.tfrecords  Img-r06.tfrecords  Img-r08.tfrecords
Img-r03.tfrecords  Img-r05.tfrecords  Img-r07.tfrecords  Img-r09.tfrecords


## 3. Training StyleGAN2-ADA

'StyleGAN2-ADA' 는 NVIDIA의 'NVlabs'라는 github를 통해 배포되어 있으며,
'FFHQ' 비롯한 고화질 이미지로 구성한 다양한 데이터셋을 학습시킨 사전 모델을 제공하고 있습니다.
(https://github.com/NVlabs/stylegan2-ada#external-data-repository)

In [ ]:
# Download Pretrained model
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl 

--2022-01-16 06:06:48--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 52.222.174.129, 52.222.174.55, 52.222.174.17, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|52.222.174.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646055 (364M) [binary/octet-stream]
Saving to: ‘ffhq.pkl’

ffhq.pkl            100%[===================>] 363.97M  69.6MB/s    in 5.2s    

2022-01-16 06:06:54 (69.4 MB/s) - ‘ffhq.pkl’ saved [381646055/381646055]



### 모델 학습은 `'train.py'`라는 파일을 사용해 진행할수 있으며, 사용법에 대한 설명 다음과 같습니다.

In [ ]:
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--metrics LIST]
                [--metricdata PATH]
                [--cfg {auto,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline}]
                [--gamma FLOAT] [--kimg INT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional arguments:
  -h, --help            show this help message and exit

general options:
  --outdir DIR          Where to save the results (required)
  --gpus INT            Number of GPUs to use (default: 1 gpu)
  --sn

### Training

In [ ]:
!pwd

/content/stylegan2-ada


In [ ]:
outdir = "/content/drive/MyDrive/styleGAN/result" # 생성된 모델이 저장될 경로

snap = 10 # 스냅샷 생성 주기('ticks'라는 회수에 기반)

res = 512 # 생성될 이미지의 기준 픽셀 수

kimg = 10000 # 훈련 (이미지) 길이 정의

augpipe = "bgcfnc" # 이미지 확대 방식 조정('blit', 'geom', 'color', 'filter', 'noise', 'cutout' or 이들의 조합)

# metrics = None # 어떤 평가 기준을 사용하여 성능을 측정할 지 정의 (default : fid50k_full)

"""
# 사전 모델의 경로를 정의, 중도에 끊기더라도 아래와 같이 생성된 모델의 경로를 대입하여, 모델 학습을 이어서 진행할 수 있습니다.
resume = "/content/drive/MyDrive/styleGAN/result/00002-Img-res512-auto1-bgcfnc-resumecustom/network-snapshot-000240.pkl"
"""
resume = "ffhq.pkl"

In [ ]:
!python train.py --outdir={outdir} --snap={snap} --data={path_dataset} \
--augpipe={augpipe} --res={res} --kimg={kimg} --resume={resume}

tcmalloc: large alloc 4294967296 bytes == 0x5611dafae000 @  0x7fd014616001 0x7fd01185954f 0x7fd0118a9b58 0x7fd0118adb17 0x7fd01194c203 0x5611d2fc2544 0x5611d2fc2240 0x5611d3036627 0x5611d3030ced 0x5611d2fc448c 0x5611d3005159 0x5611d30020a4 0x5611d2fc2d49 0x5611d303694f 0x5611d30309ee 0x5611d2f02e2b 0x5611d3032fe4 0x5611d30309ee 0x5611d2f02e2b 0x5611d3032fe4 0x5611d3030ced 0x5611d2f02e2b 0x5611d3032fe4 0x5611d2fc3afa 0x5611d3031915 0x5611d30309ee 0x5611d30306f3 0x5611d30fa4c2 0x5611d30fa83d 0x5611d30fa6e6 0x5611d30d2163
tcmalloc: large alloc 4294967296 bytes == 0x5612dafae000 @  0x7fd0146141e7 0x7fd01185946e 0x7fd0118a9c7b 0x7fd0118aa35f 0x7fd01194c103 0x5611d2fc2544 0x5611d2fc2240 0x5611d3036627 0x5611d30309ee 0x5611d2fc3bda 0x5611d3032737 0x5611d30309ee 0x5611d2fc3bda 0x5611d3032737 0x5611d30309ee 0x5611d2fc3bda 0x5611d3032737 0x5611d2fc3afa 0x5611d3031915 0x5611d30309ee 0x5611d2fc3bda 0x5611d3035d00 0x5611d30309ee 0x5611d2fc3bda 0x5611d3032737 0x5611d3030ced 0x5611d2fc448c 0x5611d300

In [ ]:
!ls -l {outdir}

total 12
drwx------ 2 root root 4096 Jan 10 05:02 00003-Img-res512-auto1-bgcfnc-resumecustom
drwx------ 2 root root 4096 Jan 16 06:53 00004-Img-res512-auto1-kimg10000-bgcfnc-resumecustom
drwx------ 2 root root 4096 Jan 16 11:08 00005-Img-res512-auto1-kimg10000-bgcfnc-resumecustom


In [ ]:
ls -l {outdir}/00005-Img-res512-auto1-kimg10000-bgcfnc-resumecustom

total 834245
-rw------- 1 root root     10210 Jan 16 11:03 events.out.tfevents.1642317809.b662e459645e
-rw------- 1 root root  40266390 Jan 16 06:56 fakes000000.png
-rw------- 1 root root  38725753 Jan 16 10:21 fakes000040.png
-rw------- 1 root root  40261998 Jan 16 06:55 fakes_init.png
-rw------- 1 root root      7821 Jan 16 11:03 log.txt
-rw------- 1 root root       139 Jan 16 10:45 metric-fid50k_full.txt
-rw------- 1 root root 351358684 Jan 16 06:56 network-snapshot-000000.pkl
-rw------- 1 root root 351358684 Jan 16 10:21 network-snapshot-000040.pkl
-rw------- 1 root root  32271619 Jan 16 06:54 reals.png
-rw------- 1 root root      2126 Jan 16 06:53 training_options.json


## Reference
- 코드 작성 및 모델 훈련은 아래의 링크들을 참고하여 진행되었습니다.
  - https://medium.com/codex/how-to-train-stylegan2-ada-in-colab-using-instagram-images-7ff552667a20
  - https://colab.research.google.com/github/Hephyrius/Stylegan2-Ada-Google-Colab-Starter-Notebook/blob/main/Stylegan2_Ada_Colab_Starter.ipynb#scrollTo=Kp0lejMIAd8i